# **Importing All necessary libraries**

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score,f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

In [3]:
def read_pickle(file_name):
    """...."""
    with open(file_name , 'rb') as handle:
        return pickle.load(handle)

In [4]:
def getScores(model_,X_train_, y_train_, X_val_, y_val_, y_pred_):
    """
    This method gets all the needed scores (accuracy, precision, recall and f1) 
    from the training/validation sets after getting the model, x train and
    validation, y train and validation, and the y predicted
    """
    print("Training Score ",model_.score(X_train_, y_train_))
    print("Validation Score ",model_.score(X_val_, y_val_))
    print("Precision: {:6.4f},   Recall: {:6.4f}, f1: {:6.4f}".format(precision_score(y_val_, y_pred_, average='micro'), 
                                                         recall_score(y_val_, y_pred_, average='micro'),f1_score(y_val_,y_pred_,average='micro')))

# **Load Datasets**

In [6]:
data = read_pickle('/content/Resumes.pickle')
data2 = read_pickle('/content/Resumes_PDF.pickle')

In [7]:
data.rename(columns = {'hospitality': 'Hospitality'}, inplace = True)

In [8]:
data4 = data2['Accounting']
data4['Accounting2'] = data4.sum(axis=1)  
data2.drop('Accounting', axis = 1, inplace = True)
data2['Accounting'] = data4['Accounting2']
data2['Business_Administration'] = 0
data2['Management_Information_System'] = 0
data2['Risk_Management'] = 0
data2['Law'] = 0
data2['Graphics_Design'] = 0
data2['Teaching'] = 0
data2['Development'] = 0
data2

<ipython-input-8-67ecff31350a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4['Accounting2'] = data4.sum(axis=1)


,Finance,Hospitality,Electrical_and_Mechanical_Engineering,Investment,Beauty_Artist,Sales_and_Marketing,Others,Secretarial,Health,Topic_Name,Accounting,Business_Administration,Management_Information_System,Risk_Management,Law,Graphics_Design,Teaching,Development
0,0.020622,0.003994,0.026564,0.000105,0.001173,0.035573,0.012777,0.013299,0.002321,Sales_and_Marketing,0.001799,0,0,0,0,0,0,0
1,0.003947,0.039175,0.009228,0.000000,0.000000,0.000000,0.000000,0.003346,0.003228,Hospitality,0.002189,0,0,0,0,0,0,0
2,0.001116,0.009441,0.003554,0.002858,0.000000,0.013625,0.000000,0.000000,0.000000,Sales_and_Marketing,0.005417,0,0,0,0,0,0,0
3,0.003965,0.047902,0.013113,0.019998,0.011939,0.002183,0.000000,0.001249,0.010942,Hospitality,0.001284,0,0,0,0,0,0,0
4,0.009024,0.047046,0.018793,0.014433,0.000000,0.000000,0.000000,0.000000,0.000964,Hospitality,0.003619,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9769,0.000000,0.036788,0.041363,0.000000,0.028259,0.000000,0.000000,0.005764,0.000000,Electrical_and_Mechanical_Engineering,0.000000,0,0,0,0,0,0,0
9770,0.000000,0.003772,0.000000,0.000000,0.003089,0.000000,0.000000,0.000000,0.000000,Hospitality,0.000000,0,0,0,0,0,0,0
9771,0.008722,0.040875,0.010314,0.024902,0.003389,0.000000,0.000000,0.008236,0.000000,Hospitality,0.022841,0,0,0,0,0,0,0
9772,0.000000,0.046050,0.000000,0.000000,0.017894,0.000102,0.000000,0.003884,0.000000,Hospitality,0.024098,0,0,0,0,0,0,0


In [9]:
data['Others'] = 0
data['Investment'] = 0
data['Beauty_Artist'] = 0
data['Secretarial'] = 0
data

,Business_Administration,Sales_and_Marketing,Accounting,Management_Information_System,Risk_Management,Law,Electrical_and_Mechanical_Engineering,Health,Graphics_Design,Finance,Teaching,Development,Hospitality,Topic_Name,Others,Investment,Beauty_Artist,Secretarial
0,0.000000,0.008697,0.000000,0.014055,0.006653,0.003310,0.024989,0.003032,0.090258,0.004661,0.004098,0.000000,0.064446,Graphics_Design,0,0,0,0
1,0.000000,0.005236,0.000000,0.007929,0.002672,0.000000,0.000000,0.000000,0.010216,0.000000,0.002681,0.000000,0.007407,Graphics_Design,0,0,0,0
2,0.000957,0.002615,0.000641,0.001609,0.003088,0.000000,0.002199,0.000000,0.035679,0.025605,0.000000,0.024663,0.015660,Graphics_Design,0,0,0,0
3,0.000674,0.018533,0.001030,0.041819,0.002782,0.000000,0.001742,0.000000,0.055123,0.000000,0.014771,0.005212,0.095702,hospitality,0,0,0,0
4,0.000000,0.000000,0.000000,0.000962,0.000000,0.000000,0.000000,0.000000,0.019586,0.000000,0.000000,0.000000,0.000000,Graphics_Design,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,0.000000,0.000900,0.000000,0.000000,0.026945,0.014721,0.006585,0.000000,0.000000,0.017832,0.005155,0.048813,0.000000,Development,0,0,0,0
1253,0.001167,0.000000,0.000000,0.000000,0.000000,0.000287,0.000000,0.000000,0.000000,0.012673,0.000000,0.122075,0.000000,Development,0,0,0,0
1254,0.000786,0.000000,0.000000,0.000000,0.000000,0.001056,0.000000,0.000000,0.000000,0.013257,0.000000,0.170425,0.000000,Development,0,0,0,0
1255,0.000000,0.011364,0.000000,0.000000,0.036696,0.000000,0.000000,0.001671,0.000000,0.000000,0.000000,0.060086,0.000000,Development,0,0,0,0


In [10]:
data = pd.concat([data, data2])

In [11]:
data.index = range(0, 11031)
data

,Business_Administration,Sales_and_Marketing,Accounting,Management_Information_System,Risk_Management,Law,Electrical_and_Mechanical_Engineering,Health,Graphics_Design,Finance,Teaching,Development,Hospitality,Topic_Name,Others,Investment,Beauty_Artist,Secretarial
0,0.000000,0.008697,0.000000,0.014055,0.006653,0.00331,0.024989,0.003032,0.090258,0.004661,0.004098,0.000000,0.064446,Graphics_Design,0.0,0.000000,0.000000,0.000000
1,0.000000,0.005236,0.000000,0.007929,0.002672,0.00000,0.000000,0.000000,0.010216,0.000000,0.002681,0.000000,0.007407,Graphics_Design,0.0,0.000000,0.000000,0.000000
2,0.000957,0.002615,0.000641,0.001609,0.003088,0.00000,0.002199,0.000000,0.035679,0.025605,0.000000,0.024663,0.015660,Graphics_Design,0.0,0.000000,0.000000,0.000000
3,0.000674,0.018533,0.001030,0.041819,0.002782,0.00000,0.001742,0.000000,0.055123,0.000000,0.014771,0.005212,0.095702,hospitality,0.0,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000962,0.000000,0.00000,0.000000,0.000000,0.019586,0.000000,0.000000,0.000000,0.000000,Graphics_Design,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11026,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.041363,0.000000,0.000000,0.000000,0.000000,0.000000,0.036788,Electrical_and_Mechanical_Engineering,0.0,0.000000,0.028259,0.005764
11027,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003772,Hospitality,0.0,0.000000,0.003089,0.000000
11028,0.000000,0.000000,0.022841,0.000000,0.000000,0.00000,0.010314,0.000000,0.000000,0.008722,0.000000,0.000000,0.040875,Hospitality,0.0,0.024902,0.003389,0.008236
11029,0.000000,0.000102,0.024098,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046050,Hospitality,0.0,0.000000,0.017894,0.003884


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
import torch

In [13]:
# Prepare the data
X = data.drop('Topic_Name', axis=1).values
y = data['Topic_Name'].values

In [14]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, RandomSampler, DataLoader

In [15]:
X = data.drop('Topic_Name', axis=1).astype(str).apply(' '.join, axis=1).values.tolist()
y = data['Topic_Name'].astype(str).values.tolist()

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded_inputs = tokenizer(X, padding=True, truncation=True, return_tensors='pt', max_length=512)

In [17]:
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

In [18]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)

In [19]:
from sklearn.model_selection import train_test_split

train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    input_ids, encoded_labels, test_size=0.2, random_state=42)
train_masks, test_masks, _, _ = train_test_split(
    attention_mask, encoded_labels, test_size=0.2, random_state=42)

In [20]:
train_inputs = torch.tensor(train_inputs).clone().detach()
test_inputs = torch.tensor(test_inputs).clone().detach()
train_masks = torch.tensor(train_masks).clone().detach()
test_masks = torch.tensor(test_masks).clone().detach()

<ipython-input-20-5152e2340347>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_inputs).clone().detach()
<ipython-input-20-5152e2340347>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_inputs = torch.tensor(test_inputs).clone().detach()
<ipython-input-20-5152e2340347>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_masks = torch.tensor(train_masks).clone().detach()
<ipython-input-20-5152e2340347>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().d

In [21]:
classes = data['Topic_Name'].unique()
num_classes = len(classes)
print(classes)
print(num_classes)

['Graphics_Design' 'hospitality' 'Teaching'
 'Electrical_and_Mechanical_Engineering' 'Sales_and_Marketing' 'Law'
 'Development' 'Health' 'Risk_Management' 'Finance'
 'Management_Information_System' 'Business_Administration' 'Accounting'
 'Hospitality' 'Beauty_Artist' 'Secretarial' 'Investment' 'Others']
18


In [22]:
# Load BERT model

from transformers import BertForSequenceClassification

num_classes = 18
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [23]:
from torch.utils.data import TensorDataset, DataLoader

# Convert variables to torch.Tensor if needed
train_inputs = torch.tensor(train_inputs)
train_masks = torch.tensor(train_masks)
train_labels = torch.tensor(train_labels)

# Create a TensorDataset
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)

# Define batch size
batch_size = 16

# Create the data loader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

<ipython-input-23-3751764cea00>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_inputs)
<ipython-input-23-3751764cea00>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_masks = torch.tensor(train_masks)


In [24]:
from transformers import AdamW, get_linear_schedule_with_warmup

# Define the learning rate
learning_rate = 2e-5
num_epochs = 10

optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
warmup_steps = int(0.1 * total_steps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
# training loop
max_grad_norm = 1.0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [26]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()

    average_loss = total_loss / len(train_dataloader)
    print(f'Epoch: {epoch+1}, Average Loss: {average_loss}')

Epoch: 1, Average Loss: 2.211151273786158
Epoch: 2, Average Loss: 1.149903226022919
Epoch: 3, Average Loss: 0.5371222501971583
Epoch: 4, Average Loss: 0.3396097765358138
Epoch: 5, Average Loss: 0.254713113215563
Epoch: 6, Average Loss: 0.1794012249945123
Epoch: 7, Average Loss: 0.1532730115311848
Epoch: 8, Average Loss: 0.11625074985246445
Epoch: 9, Average Loss: 0.08886616901503842
Epoch: 10, Average Loss: 0.07218202252145114


In [30]:
# Assuming you have loaded the test inputs, test masks, and test labels correctly
test_inputs = torch.tensor(test_inputs)
test_masks = torch.tensor(test_masks)
test_labels = torch.tensor(test_labels)

# Create the test dataset
test_dataset = TensorDataset(test_inputs, test_masks, test_labels)

# Create the test data loader
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

<ipython-input-30-386d8521e34d>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_inputs = torch.tensor(test_inputs)
<ipython-input-30-386d8521e34d>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_masks = torch.tensor(test_masks)


In [31]:
model.eval()
total_correct = 0
total_samples = 0

In [32]:
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'labels': batch[2]}
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    _, predicted_labels = torch.max(logits, dim=1)
    total_correct += (predicted_labels == inputs['labels']).sum().item()
    total_samples += len(inputs['labels'])

accuracy = total_correct / total_samples
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.9646579066606252


In [34]:
from tabulate import tabulate

# Define the performance metrics
performance_data = [
    ["BERT Model Performance"],
    ["Metric", "Value"],
    ["Training Loss", average_loss],
    ["Test Accuracy", accuracy],
]

# Display the table
print(tabulate(performance_data, headers="firstrow", tablefmt="fancy_grid"))

╒═══════════════╤══════════════════════════╕
│               │ BERT Model Performance   │
╞═══════════════╪══════════════════════════╡
│ Metric        │ Value                    │
├───────────────┼──────────────────────────┤
│ Training Loss │ 0.07218202252145114      │
├───────────────┼──────────────────────────┤
│ Test Accuracy │ 0.9646579066606252       │
╘═══════════════╧══════════════════════════╛
